In [16]:
import pandas as pd
import error_prompts as p
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()


prompt_name_structured = 'structured_analysis'
prompt_structured = p.structured_analysis
prompt_name_unstructured = 'unstructured_analysis'
prompt_unstructured = p.unstructured_analysis

file = "dynamic_fewshot_multi_table_gpt-4-turbo-preview"

DATA_PATH = f'../../Datasets/Evaluations/Schema Matching/{file}.csv'
OUTPUT_PATH = f"../../Datasets/Evaluations/Schema Matching/Error_Analysis/structured_unstructured/unstructured_{file}.csv"

MODEL = "gpt-4-turbo-preview"
TEMP = 0





In [17]:
df = pd.read_csv(DATA_PATH)
print(df.head())

   Unnamed: 0  table_index                      table_name  column_index_left  \
0           0            0  29021592_3_2299138476894681059                  0   
1           1            1  29021592_3_2299138476894681059                  3   
2           2            2  29021592_3_2299138476894681059                  4   
3           3            3  29021592_3_2299138476894681059                  2   
4           4            4  29021592_3_2299138476894681059                  0   

   column_index_right  y_true  y_pred                  prompt_name  \
0                   0    True    True  dynamic_fewshot_multi_table   
1                   0   False   False  dynamic_fewshot_multi_table   
2                   0   False   False  dynamic_fewshot_multi_table   
3                   0   False   False  dynamic_fewshot_multi_table   
4                   3   False   False  dynamic_fewshot_multi_table   

                                              prompt  \
0  Human: Example 1: \nInput:\nTable

In [19]:

df['error'] = None
df['error'] = ['correct' if y_true == y_pred else 'error' for y_true, y_pred in zip(df['y_true'], df['y_pred'])]

print(df['error'].value_counts())

df_correct = df[df['error'] == 'correct']
df_error = df[df['error'] == 'error']



df_error_sample = df_error.sample(n=20)
df_correct_sample = df_correct.sample(n=5)
df = pd.concat([df_error_sample, df_correct_sample])
df['structured_analysis'] = None
df['unstructured_analysis'] = None
df = df.reset_index(drop=True)




error
error      20
correct     5
Name: count, dtype: int64


In [20]:
print(df.shape)
print(df.head())

(25, 19)
   Unnamed: 0  table_index                      table_name  column_index_left  \
0         235          235  61121469_0_6337620713408906340                  0   
1         168          168  36039980_4_4283009829602711082                  4   
2          94           94  84548468_0_5955155464119382182                  4   
3          44           44  55961337_0_6548713781034932742                  3   
4         408          408  49801939_0_6964113429298874283                  3   

   column_index_right  y_true  y_pred                  prompt_name  \
0                   0    True   False  dynamic_fewshot_multi_table   
1                   0    True   False  dynamic_fewshot_multi_table   
2                   0    True   False  dynamic_fewshot_multi_table   
3                   4    True   False  dynamic_fewshot_multi_table   
4                   3    True   False  dynamic_fewshot_multi_table   

                                              prompt  \
0  Human: Example 1: \nInpu

In [21]:
#unstructured Analysis
def execute(df, prompt, start = 0):
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=100)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=100)
    for i in range(len(df)):
        if(df["unstructured_analysis"][i] != None and df["unstructured_analysis"][i] != "error"):
            continue
        print(i)
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        user_prompt = df.loc[i, 'prompt']
        ai_answer = str(df.loc[i, 'ai_answer'])
        match = ""
        if str(df.loc[i, 'y_pred']).lower() == 'false':
            match = "not match"
        else:
            match = "match"
        columns = "Column A-" + str(df.loc[i, 'column_index_left']) + " and Column B-" + str(df.loc[i, 'column_index_right'])
        try: 
            result = chain.run(user_prompt = user_prompt, ai_answer = ai_answer, match = match, columns = columns, callbacks=[handler])
        except Exception as e:
            print(e)
            print(chain.prompt.format_prompt(user_prompt = user_prompt, ai_answer = ai_answer, match = match, columns = columns).to_string())
            df.loc[i, 'unstructured_analysis'] = 'error'
            continue
        handler.langfuse.flush()
        prompt_text = chain.prompt.format_prompt(user_prompt = user_prompt, ai_answer = ai_answer, match = match, columns = columns).to_string()
        if (i < 5):
            print(prompt_text)
            print("\n")
            print(result)
            print("\n")
        
        if (i % 50 == 0):
            print(str(i) + " of  " + str(len(df)))
        df.loc[i, 'unstructured_analysis']= result
    return df


df = execute(df, prompt_unstructured)

    



0
System: You are a helpful AI.
Human: Human: Example 1: 
Input:
Table A: 
|   Column A-0 | Column A-1                  | Column A-2                                                                                                                                   | Column A-3         |   Column A-4 |
|-------------:|:----------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------|:-------------------|-------------:|
|         1988 | Federal Republic of Germany | "for their determination of the three-dimensional structure of a photosynthetic reaction centre"[88]                                         | Johann Deisenhofer |          nan |
|         1923 | Austria                     | "for his invention of the method of micro-analysis of organic substances"[28]                                                                | Fritz Pregl        |          nan |
|         1910 | Germ

In [15]:
df.to_csv(OUTPUT_PATH, index=False)